# Repeat quasars

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from astropy.table import Table, join, hstack, Column, unique
from astropy.io import ascii

 ### Query the data via `bossquery` from the bossdata package:
 #### http://bossdata.readthedocs.org/en/latest/scripts.html#bossquery

#### BOSS 

Pipeline version number: export BOSS_REDUX_VERSION=v5_7_0

`bossquery --full --what 'PLATE,MJD,FIBER,THING_ID,RUN,RERUN,CAMCOL,FIELD,ID' --where 'CLASS="QSO" and ZWARNING=0 and Z_ERR>0 and RUN>0' --max-rows 400000 --save boss_qsos.dat`

#### DR13 (BOSS+SEQUELS)

Pipeline version number: export BOSS_REDUX_VERSION=v5_9_0

`bossquery --full --what 'PLATE,MJD,FIBER,THING_ID,RUN,RERUN,CAMCOL,FIELD,ID' --where 'CLASS="QSO" and ZWARNING=0 and Z_ERR>0 and EBOSS_TARGET1&4096>0' --max-rows 100000 --save dr13_qsos.dat` 

##### Note: This query return no rows

#### eBOSS 

Pipeline version number: export BOSS_REDUX_VERSION=v5_9_1

`bossquery --full --what 'PLATE,MJD,FIBER,THING_ID,RUN,RERUN,CAMCOL,FIELD,ID' --where 'CLASS="QSO" and ZWARNING=0 and Z_ERR>0 and RUN>0 EBOSS_TARGET1&4096>0' --max-rows 100000 --save eboss_qsos.dat`

In [14]:
boss = ascii.read('boss_qsos.dat')
eboss = ascii.read('eboss_qsos.dat', format='csv', delimiter=' ')

In [15]:
print 'eBOSS returned %d quasars' %len(eboss)
print 'BOSS returned %d quasars' %len(boss)

eBOSS returned 9851 quasars
BOSS returned 340202 quasars


### Match quasars

In [16]:
overlap = join(eboss, boss, keys=('RUN','RERUN','CAMCOL','FIELD', 'ID'), join_type='inner', table_names=('E','B'))
overlap = overlap.group_by(('PLATE_E','MJD_E','FIBER_E'))
repeats = Table([overlap['PLATE_E'],overlap['MJD_E'],overlap['FIBER_E'],overlap['THING_ID_E'],overlap['PLATE_B'],overlap['MJD_B'],overlap['FIBER_B'],overlap['THING_ID_B']])

In [17]:
repeats

PLATE_E,MJD_E,FIBER_E,THING_ID_E,PLATE_B,MJD_B,FIBER_B,THING_ID_B
int64,int64,int64,int64,int64,int64,int64,int64
7572,56944,5,267246461,5028,55836,149,267490850
7572,56944,5,267246461,5029,55750,498,267490850
7572,56944,226,272920497,5028,55836,653,273165985
7572,56944,348,270126146,5020,55852,78,270370986
7572,56944,504,277500052,5022,55827,206,277746813
7572,56944,504,277500052,5020,55852,790,277746813
7572,56944,520,276061714,5020,55852,838,276307924
7572,56944,526,275822186,5020,55852,800,276068348
7572,56944,648,277136879,5020,55852,916,277383514


In [ ]:
#repeats.write('eboss_boss_repeat_qsos.dat', format='ascii')

In [7]:
eboss_overlap = Table([repeats['PLATE_E'],repeats['MJD_E'],repeats['FIBER_E'],repeats['THING_ID_E']])
unique_eboss = unique(eboss_overlap)

In [ ]:
plt.figure()
plt.hist(eboss['MJD'], bins=np.linspace(56800, 57250, 60))
plt.hist(unique_eboss['MJD_E'], bins=np.linspace(56800, 57250, 60));

In [13]:
#Around 98% of eBOSS quasars are reobserved BOSS quasars (when overlap is performed over 340,202 BOSS quasars)
float(len(unique_eboss))/len(eboss)

0.984468581869861